In [19]:
import numpy as np
import pandas as pd

### 数据预处理

数据来源

http://www.nhc.gov.cn/

In [20]:
Data_China=pd.read_csv('./dataset/中国前期疑似&确诊.csv')

In [21]:
Data_China.head()

,日期,确诊,疑似,医学观察
0,2020/1/11,41,NaN,739.0
1,2020/1/12,41,NaN,717.0
2,2020/1/13,41,NaN,687.0
3,2020/1/14,41,NaN,576.0
4,2020/1/15,41,NaN,313.0


- 数据清理

In [22]:
Data_China['确诊']=Data_China['确诊'].fillna(0)
Data_China['疑似']=Data_China['疑似'].fillna(0)
Data_China['医学观察']=Data_China['医学观察'].fillna(0)

In [23]:
Data_China=Data_China[:-1]
Data_China.head()

,日期,确诊,疑似,医学观察
0,2020/1/11,41,0.0,739.0
1,2020/1/12,41,0.0,717.0
2,2020/1/13,41,0.0,687.0
3,2020/1/14,41,0.0,576.0
4,2020/1/15,41,0.0,313.0


In [24]:
from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio
#setting offilne
plotly.offline.init_notebook_mode(connected=True)

- 可视化

In [25]:
fig = go.Figure(data=[
    go.Bar(name='确诊', x=Data_China['日期'], y=Data_China['确诊']),
    go.Bar(name='疑似', x=Data_China['日期'], y=Data_China['疑似']),
    go.Bar(name='医学观察', x=Data_China['日期'], y=Data_China['医学观察'])
])
# Change the bar mode
fig.update_layout(barmode='stack',template='seaborn',title='我国疫情爆发早期确诊&疑似(医学观察)病例')
fig.show()

#### SEIR

《人民日报》初期报导提到59个疑似病例有41位最终确诊，因此ρ的一个参考值为：
ρ=41⁄59=0.695 


其中S(t)、L(t)、I(t)和R(t)分别表示t时刻网络中处于易感染态、潜伏态、感染态和恢复态个体数目，N表示网络中个体总数目，且N=S(t)+L(t)+I(t)+R(t)。当t趋近于0时，有S(t)趋近于N，基本再生数可表示为

潜伏期和感染期可分别表示为：
T_L=1/γ_1 和T_I=1/γ_2   


对于生成时间Tg,文献[6]基于SARS在新加坡爆发的分析，认为Tg均值是8.4天，但传播爆发的早期(前两周)均值为10天，基于文献[1]对武汉肺炎少量病例的分析，Tg均值恰为8.4天。因为病毒传播系数对Tg的值比较敏感，所以Tg的取值为：
T_g=8.4 & T_g=10              


In [26]:
p = 0.695

In [27]:
defi=Data_China['确诊']
susp=Data_China['疑似']
t = Data_China['日期']

In [28]:
Tg_1 = 8.4
Tg_2 = 10
from math import *

In [29]:
R01=[]
R02=[]
for i in range(len(defi)):
    yt=susp[i]*p+defi[i] # 真实值
    lamda =log(yt)/(i+35) #此处取距离疫情开始爆发 34天
    R0_1 = 1 + lamda * Tg_1 + p * (1 - p) * (math.pow((lamda * Tg_1),2))
    R0_2 = 1 + lamda * Tg_2 + p * (1 - p) * (math.pow((lamda * Tg_2),2))
    R01.append(R0_1)
    R02.append(R0_2)

- 找到最大值

In [30]:
R01_max=R01.index(max(R01))
R02_max=R02.index(max(R02))
R01_min=R01.index(min(R01))
R02_min=R02.index(min(R02))

In [31]:
data=[]
trace0=go.Scatter(
        x = t,
        y = R01,
        mode = 'lines+markers',
        name = 'Tg=8.4',
        marker = dict(color = 'rgb(166,206,227)')
)
trace1=go.Scatter(
        x = t,
        y = R02,
        mode = 'lines+markers',
        name = 'Tg=10',
        marker = dict(color = 'rgb(251,154,153)'),
)
data.append(trace0)
data.append(trace1)
layout=go.Layout(
    showlegend=True,
    annotations=[
        dict(
            x=R01_max,
            y=max(R01),
            xref="x",
            yref="y",
            text=str(format(max(R01),'.4f')),
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=30
        ),
        dict(
            x=R01_min,
            y=min(R01),
            xref="x",
            yref="y",
            text=str(format(min(R01),'.4f')),
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-20
        ),
        dict(
            x=R02_min,
            y=min(R02),
            xref="x",
            yref="y",
            text=str(format(min(R02),'.4f')),
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-30
        ),
        dict(
            x=R02_max,
            y=max(R02),
            xref="x",
            yref="y",
            text=str(format(max(R02),'.4f')),
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=30
        )
    ],
    template='ggplot2',
    title='COVID-19 病毒传播系数变化趋势',
    xaxis= dict(title= '日期',ticklen= 5,zeroline= False), 
    yaxis= dict(title= '病毒传播系数',ticklen= 5,zeroline= False)
)
fig=go.Figure(data=data,layout=layout)
py.iplot(fig)